In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import layers
import os

from process_ntuples import *

In [194]:
dir_path = "/global/cfs/cdirs/atlas/gottino/Charm/Analysis/run_batch_gup/ntuples"

files = os.listdir(dir_path)
TTree = process_ntuple(dir_path, files[0])
files

['CharmAnalysis.mc.ForcedDecay.Dplus.tree.root',
 'CharmAnalysis.mc.ForcedDecay.LabmdaC.tree.root']

In [288]:
DMesons_df = show(0, TTree, ["DMesons_daughterInfo__pt", "DMesons_daughterInfo__eta", "DMesons_daughterInfo__phi", "DMesons_daughterInfo__pdgId"], 100)
DMesons_df = remove_empty(DMesons_df)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

DMesons_df

,DMesons_daughterInfo__pt,DMesons_daughterInfo__eta,DMesons_daughterInfo__phi,DMesons_daughterInfo__pdgId
0,"((1118.3367, 3805.8997, 971.6052), (3805.8997,...","((1.0579262, 0.8759657, 1.0816675), (0.8759657...","((1.2175969, 1.3583351, 0.71336174), (1.358335...","((-321, 211, 211), (2212, -321, 211))"
1,"((6094.893, 1307.462, 2733.4648))","((0.15998553, 0.011107716, -0.0614343))","((2.6128821, 2.7195172, 2.9228168))","((-321, 211, 211))"
2,"((6457.058, 5970.0933, 6637.575), (6605.627, 6...","((-1.608788, -1.6307458, -1.7315177), (-1.7326...","((-0.371787, -0.5164238, -0.4971406), (-0.4966...","((-321, 211, 211), (2212, -321, 211))"
3,"((3794.4978, 3308.3105, 3065.4814), (3308.3105...","((-0.073717244, 0.22605924, -0.017923404), (0....","((0.753864, 0.6827104, 0.920779), (0.6827104, ...","((-321, 211, 211), (2212, -321, 211))"
4,"((3781.5564, 7396.4673, 1412.3259), (7396.4673...","((-0.9284433, -1.182188, -1.1012387), (-1.1821...","((0.8518817, 0.7419038, 0.77916163), (0.741903...","((-321, 211, 211), (2212, -321, 211))"
...,...,...,...,...
60,"((4757.746, 1190.8936, 6516.625), (6508.5923, ...","((-1.1418481, -1.4339805, -1.0030863), (-1.002...","((0.94814, 1.024492, 0.8848141), (0.88491267, ...","((-321, 211, 211), (2212, -321, 211))"
61,"((9512.122, 6988.785, 4322.3135))","((-0.25393596, -0.1254045, -0.22442578))","((-1.2898706, -1.1826429, -1.1917845))","((2212, -321, 211))"
62,"((17158.73, 11640.837, 2144.3984), (11640.837,...","((1.05311, 0.96944314, 1.085608), (0.96944314,...","((-1.2230655, -1.158654, -1.24622), (-1.158654...","((-321, 211, 211), (2212, -321, 211))"
63,"((2697.8123, 2267.6973, 1967.2911))","((0.5633415, 0.8389925, 0.6217043))","((-1.2840899, -1.0002437, -1.5307286))","((2212, -321, 211))"


In [289]:
DPlus = [-321, 211, 211]
LambdaC = [2212, -321, 211]

remove_lambda = []

count = 0
for index, row in DMesons_df.iterrows():
    if count == 50:
        break
    count+=1
    
    pts = row['DMesons_daughterInfo__pt']
    etas = row['DMesons_daughterInfo__eta']
    phis = row['DMesons_daughterInfo__phi']
    pdgids = row["DMesons_daughterInfo__pdgId"]
    
    #loop through until the first lambda encountered
    for i in range(len(pdgids) - 1):
        #if there are multiple lambda baryons
        if len(pdgids) > 2:
            #print(np.shape(pdgids))
            #stop at the first lambda and keep the higher proton pt valued lambda
            if list(pdgids[i]) == LambdaC:
                l1_pt, l2_pt = pts[i][0], pts[i + 1][0]
                if (l1_pt > l2_pt):
                    #Lambda 1's proton has higher pt
                    remove_lambda.append((index, i + 1))
                elif (l1_pt < l2_pt):
                    #Lambda 2's proton has higher pt!
                    remove_lambda.append((index, i))
                else:
                    raise Exception("Error: Are the proton pts equal?")

In [297]:
#takes tuples of shape (2,) / (index, column)
def remove_particles_from_df(df, tuple_list):
    for tup in tuple_list:
        index, column = tup
        print(index, column)
        
        new_df = df.copy()
        #new_df["DMesons_daughterInfo__pt"] = new_df["DMesons_daughterInfo__pt"].apply(lambda x: print(x))
        for i in range(len(DMesons_df.iloc[index, 0][column]) + 1):
            #new_df["DMesons_daughterInfo__pt"] = new_df["DMesons_daughterInfo__pt"].apply(lambda x: print(x))
            value = new_df.iloc[index, i][column]
            print("Value: ",value)
    return new_df

df = remove_particles_from_df(DMesons_df, remove_lambda[:10])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print("-----------------")
print(DMesons_df.iloc[32])
print(DMesons_df.iloc[32, 0])
print(DMesons_df.iloc[32, 0][2])
print(DMesons_df.iloc[32, 1][2])
print(DMesons_df.iloc[32, 2][2])
print(DMesons_df.iloc[32, 3][2])
print("")


32 2
Value:  [3649.7476, 769.11566, 4525.2417]
Value:  [-2.241756, -2.1965945, -2.1482377]
Value:  [-2.6522913, -2.4690387, -2.7042263]
Value:  [2212, -321, 211]
46 1
Value:  [6202.453, 1940.8235, 6434.7905]
Value:  [-0.9160974, -0.82834303, -0.8550062]
Value:  [-0.2525417, -0.015253778, -0.17007089]
Value:  [2212, -321, 211]
48 1
Value:  [4955.2544, 2360.5442, 6293.0254]
Value:  [-1.6138946, -1.5858023, -1.4756751]
Value:  [0.82840514, 0.9031059, 0.69141525]
Value:  [2212, -321, 211]
-----------------
DMesons_daughterInfo__pt       ((769.11566, 4536.6436, 3666.318), (4536.6436,...
DMesons_daughterInfo__eta      ((-2.1965945, -2.1476398, -2.2415924), (-2.147...
DMesons_daughterInfo__phi      ((-2.4690387, -2.7047215, -2.653167), (-2.7047...
DMesons_daughterInfo__pdgId    ((-321, 211, 211), (2212, -321, 211), (2212, -...
Name: 32, dtype: object
[[769.11566, 4536.6436, 3666.318], ..., [3649.7476, 769.11566, 4525.2417]]
[3649.7476, 769.11566, 4525.2417]
[-2.241756, -2.1965945, -2.1482377]

In [81]:
model = Sequential()
model.add(tf.keras.Input(shape = (23)))
model.add(Dense(6, activation = "relu"))
model.add(Dense(1, activation = "relu"))

loss=tf.keras.losses.MeanSquaredError()
model.compile(loss= loss, optimizer= tf.keras.optimizers.Adam(learning_rate = 0.01))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 144       
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


2023-03-30 21:13:34.071910: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-30 21:13:34.072000: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cori13): /proc/driver/nvidia/version does not exist
2023-03-30 21:13:34.078976: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
